<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
# Request package will be used to download the data.
download = False # Set to True if you need to download the data

if download:
    import requests 

    url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"
    response = requests.get(url)

    # Save the file
    with open("concrete_data_week4.zip", "wb") as file:
        file.write(response.content)

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [ ]:
# Unzip the data
unzip = False # Set to True if you need to unzip the data

if unzip:
    import zipfile

    zip_file = 'concrete_data_week4.zip'
    extract_to = 'concrete_data_week4'

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print("File unzipped")

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [5]:
# 1. Import the library
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# 2. Define batch size for training and validation
batch_size = 100

In [7]:
# 3. Construct an ImageDataGenerator for both training and validation
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

reshape = 224
train_generator = data_generator.flow_from_directory(
    '.\\concrete_data_week4\\concrete_data_week4\\train',
    target_size=(reshape, reshape),
    batch_size=100, 
    class_mode='categorical'
)

valid_generator = data_generator.flow_from_directory(
    '.\\concrete_data_week4\\concrete_data_week4\\valid',
    target_size=(reshape, reshape),
    batch_size=100, 
    class_mode='categorical'
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [8]:
# 4. Create the model
num_classes = 2
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,715,714 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [9]:
# 5. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 6. Fit the model
from tensorflow.keras.models import load_model

epochs = 2
steps_per_epoch = len(train_generator)
validation_steps = len(valid_generator)

train = False # Set to True to train the model

if train:
    fit_history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=valid_generator,
        validation_steps=validation_steps,
        verbose=1,
    )
    model.save('classifier_vgg16_model.h5')
else:
    model = load_model("classifier_vgg16_model.h5")

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [13]:
# 1. Load the ResNet50 model
resnet_model = load_model('classifier_resnet_model.h5')

In [14]:
# 2. Construct an ImageDataGenerator for test set
test_data_generator = ImageDataGenerator()
test_generator = test_data_generator.flow_from_directory(
    '.\\concrete_data_week4\\concrete_data_week4\\test',
    target_size=(reshape, reshape),
    shuffle=False,
)

Found 500 images belonging to 2 classes.


In [15]:
# 3. Evaluate the model using the evaluate method
# Note: evaluate method is used instead of evaluate_generator method in newer Keras versions
vgg_test_loss, vgg_test_accuracy = model.evaluate(test_generator)
resnet_test_loss, resnet_test_accuracy = resnet_model.evaluate(test_generator)

C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 64s 4s/step - accuracy: 0.9719 - loss: 0.0670
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.9909 - loss: 0.0393


In [16]:
# 4. Print the performance of VGG16 model
print(f"VGG16 Test Loss: {vgg_test_loss:.4f}")
print(f"VGG16 Test Accuracy: {vgg_test_accuracy:.4f}")

VGG16 Test Loss: 0.1642
VGG16 Test Accuracy: 0.9300


In [20]:
# 5. Print the performance of ResNet model
print(f"ResNet Test Loss: {resnet_test_loss:.4f}")
print(f"ResNet Test Accuracy: {resnet_test_accuracy:.4f}")

ResNet Test Loss: 0.0852
ResNet Test Accuracy: 0.9780


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [18]:
# 1. Predict the class of images in the test data by using the predict method
predictions_vgg16 = model.predict(test_generator)
predictions_resnet = resnet_model.predict(test_generator)

16/16 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step


In [21]:
# 2. Report the first 5 class predictions
print("VGG16 model predictions:")
for i in range(5):
    print(f"Image {i+1} class:", "Positive" if predictions_vgg16[i].argmax() == 1 else "Negative")

print("\nResNet model predictions:")
for i in range(5):
    print(f"Image {i+1} class:", "Positive" if predictions_resnet[i].argmax() == 1 else "Negative")

VGG16 model predictions:
Image 1 class: Negative
Image 2 class: Negative
Image 3 class: Negative
Image 4 class: Negative
Image 5 class: Negative

ResNet model predictions:
Image 1 class: Negative
Image 2 class: Negative
Image 3 class: Negative
Image 4 class: Negative
Image 5 class: Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).